In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import gzip

# Change directory to the folder where the .gz file is located
os.chdir('/content/drive/MyDrive/Data/')

# Specify the path to the .gz file
gz_file_path = 'amazon-meta.txt.gz'

# Extract the .gz file
with gzip.open(gz_file_path, 'rb') as gz_file:
    with open('amazon-meta.txt', 'wb') as out_file:
        out_file.write(gz_file.read())

In [ ]:
dataset_folder = '/content/drive/MyDrive/Data/'
file_name = 'amazon-meta.txt'
file_path = dataset_folder + file_name

In [ ]:
import numpy as np
import json
import re
import pandas as pd
from pandas import json_normalize

In [ ]:
# print the first few lines of the text file
def view_file(file_path,lines=50):
  with open(file_path, 'r') as file:
    for lineNo,line in enumerate(file):
      if(lineNo>lines):
        break
      print(line)


In [ ]:
def import_from_file(filename):
    with open(filename, 'r',encoding='utf-8', errors='ignore') as f:
        data = []
        entry = {}
        last_key = None
        for num,line in enumerate(f):
            if(num<3) :
                continue
            line = line.strip()
            if line.startswith('Id:'):
                if entry:
                    if "discontinued product" not in entry[last_key]:
                        data.append(entry)
                entry = {"Id":line[3:].strip()}

            elif line:
                if not line[0].isalpha() or ':' not in line:
                    entry[last_key] += ',' + line
                else:
                    key,value = line.split(':', 1)
                    entry[key.strip()] = value.strip()
                    last_key = key

        if entry:
            data.append(entry)
    return data

def post_Processing(filedata):
    for item in filedata:

        for key,value in item.items():
            if key == 'similar':
                count, *similar = value.split()
                item[key] = {"count":count, "ASIN ID":similar}
            elif key == 'categories':
                subentry = {}
                value = value.split(',')
                subentry['count'] = value[0]
                subentry['ASIN ID'] = value[1:]
                item[key] = subentry
            elif key == 'reviews':
                pass
                subentry = {}
                value = value.split(',')

                pattern = r'(\S+):\s*(\S+)'
                indexes = re.findall(pattern,value[0])
                for index in indexes:
                    subentry[index[0]] = index[1]

                reviewList = []
                for i in range(len(value[1:])):

                    currentReview = {}
                    currentReview["Review ID"] = i+1
                    currentReview["Date"] = re.split(r'\s+', value[1:][i])[0]
                    pattern = r'(\S+):\s*(\S+)'
                    indexes = re.findall(pattern,value[1:][i])
                    for index in indexes:
                        currentReview[index[0]] = index[1]

                    reviewList.append(currentReview)

                subentry['review list'] = reviewList
                item[key] = subentry

    return filedata



In [ ]:
data = import_from_file(file_path)
data = post_Processing(data)

In [ ]:
updated_data = []
for item in data:
  if item['similar']['count'] == '5' and item['salesrank'] != '0':
    updated_data.append(item)

In [ ]:
#select only 10000 random data of updated_data
import random
random_data = random.sample(updated_data, 10000)


In [ ]:
from datetime import datetime


In [ ]:
reviews_data = []
for item in random_data:
  for review in item['reviews']['review list']:
    date_string = review['Date']

    # Convert the date string to a datetime object
    date_object = datetime.strptime(date_string, '%Y-%m-%d')

    # Extract year, month, and day from the datetime object
    year = date_object.year
    month = date_object.month
    day = date_object.day
    reviews_data.append({
      'customer id': review['cutomer'],
      'ASIN': item['ASIN'],
      'group': item['group'],
      'day': day,
      'month': month,
      'year': year
    })

In [ ]:
#store the content of reviews_data into a file
with open(dataset_folder + 'reviews_data.txt', 'w') as file:
  file.write(str(reviews_data))

In [ ]:
baseCost = { 'Book':1000, 'DVD': 800, 'Video': 500, 'Music': 200, 'Software': 2000}

In [ ]:
reviews = reviews_data

In [ ]:
#create a new column price and assign random price to each item
import random
for item in reviews:
  base = baseCost[item['group']] 
  price =  random.randint(int(base -0.50 * base), int(base + 0.50 * base))
  item['profit'] = random.randint(3,15)/100 * price


In [ ]:
!pip install pyspark

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
filtered_reviews = []
for review in reviews:
  if review['year'] == 2000 and review['month'] == 6:
    filtered_reviews.append(review)

In [ ]:
reviews = filtered_reviews

In [ ]:
# use pyspark to convert the reviews list to a table
#install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
reviews_df = spark.createDataFrame(reviews)


In [ ]:
#print the schema of reviews_df
reviews_df.printSchema()

In [ ]:
reviews_df.createOrReplaceTempView("reviews_table_june_2000")
query = "SELECT day, group, SUM(profit) as daily_profit, COUNT(*) as count FROM reviews_table_june_2000 GROUP BY day, group ORDER BY day"
reviews_count_per_day = spark.sql(query)

In [ ]:
reviews_count_per_day.show()

In [ ]:
# Select only the 'month' and 'count' columns
import matplotlib.pyplot as plt

music_reviews = reviews_count_per_day.filter(reviews_count_per_day.group == 'Music')

music_reviews_month_count = music_reviews.select('day', 'count','daily_profit')
# Collect the rows as a list of dictionaries
result_rows = music_reviews_month_count.collect()

# Extract months and counts from the rows
months = [row['day'] for row in result_rows]
counts = [row['count'] for row in result_rows]
profit = [row['daily_profit'] for row in result_rows]
# Plot the data
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot the bar chart on the left subplot (index 0)
axs[0].bar(months, counts)
axs[0].set_xlabel('day')
axs[0].set_ylabel('Count')
axs[0].set_title('Current Trend of User Interaction on platform')
# axs[0].tick_params(axis='x', rotation=45, ha='right')  # Rotate the x-axis labels for better readability

# If you want to add more subplots on the right side, you can plot them here.
# For example:
axs[1].bar(months, profit)
axs[1].set_xlabel('months')
axs[1].set_ylabel('profit')
axs[1].set_title('Current Trend of daily profit')

# Adjust the spacing between the subplots to prevent overlapping labels
plt.tight_layout()

# Show the plot
plt.show()
# In this example, the given bar chart will be plotted on the left side (index 0) of the 1x2 grid of subplots. If you want to add more subplots to the right, you can simply create additional plots by referring to axs[1], axs[2], and so on, and customize them as needed. The plt.tight_layout() function ensures that the subplots are properly spaced to prevent overlapping labels and titles.


In [ ]:

def plotData(reviews, group, year=2000, month=6):
  print(group,year,month)
  print(reviews[0])
  filtered_reviews = []
  for review in reviews:
    if review['group'] == group and review['year'] == year and review['month'] == month:
      filtered_reviews.append(review)

  # from pyspark.sql import SparkSession
  spark = SparkSession.builder.getOrCreate()
  reviews_df = spark.createDataFrame(filtered_reviews)

  reviews_df.createOrReplaceTempView("reviews_table")
  query = "SELECT day, group, SUM(profit) as daily_profit, COUNT(*) as count FROM reviews_table GROUP BY day, group ORDER BY day"
  reviews_count_per_day = spark.sql(query)

  # import matplotlib.pyplot as plt

  group_reviews = reviews_count_per_day.filter(reviews_count_per_day.group == group)

  group_reviews_month_count = group_reviews.select('day', 'count','daily_profit')
  # Collect the rows as a list of dictionaries
  result_rows = group_reviews_month_count.collect()

  # Extract months and counts from the rows
  months = [row['day'] for row in result_rows]
  counts = [row['count'] for row in result_rows]
  profit = [row['daily_profit'] for row in result_rows]
  # Plot the data
  fig, axs = plt.subplots(1, 2, figsize=(16, 6))

  # Plot the bar chart on the left subplot (index 0)
  axs[0].bar(months, counts)
  axs[0].set_xlabel('day')
  axs[0].set_ylabel('Count')
  axs[0].set_title('Current Trend of User Interaction on platform')
  # axs[0].tick_params(axis='x', rotation=45, ha='right')  # Rotate the x-axis labels for better readability

  # If you want to add more subplots on the right side, you can plot them here.
  # For example:
  axs[1].bar(months, profit)
  axs[1].set_xlabel('months')
  axs[1].set_ylabel('profit')
  axs[1].set_title('Current Trend of daily profit')

  # Adjust the spacing between the subplots to prevent overlapping labels
  plt.tight_layout()

  # Show the plot
  plt.show()

In [ ]:
def loadData():
  import csv
  reviews = []
  with open('reviews_data.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
      reviews.append(row)
  plotData(reviews,'Book',2000,1)
  return reviews

loadData()